# Sentence extraction

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
import nltk
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import string
import re
import pandas as pd
import numpy as np
from operator import itemgetter

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from nltk.chunk import RegexpParser
from nltk import sent_tokenize,word_tokenize
from itertools import combinations
pattern = """
    NP: {<JJ>*<NN>+}   
    {<JJ>*<NN><CC>*<NN>+}
    """
chunker = RegexpParser(pattern)

In [ ]:
name = pd.read_excel('name.xlsx')
name['other_name'] = name['other_name'].fillna('')
name['all_names'] = name['name'] +','+ name['other_name']
name_dict = {}
for i in range(len(name)):
    m = name['all_names'][i].split(',')
    race = name['race'][i]
    for j in range(len(m)):
        name_dict[m[j]] = [i,race]
name_list=pd.DataFrame()
name_list['code']=pd.DataFrame(list(name_dict.values()))[0]
name_list['race']=pd.DataFrame(list(name_dict.values()))[1]
name_list['name']=list(name_dict.keys())
name_list = name_list.sort_values(by='race')
name_list = name_list.sort_values(by='code')
name_list.to_excel('name_list.xlsx')

In [ ]:
name_list.columns=['name_code','race','name']

### Extract the sentences

In [ ]:
def data_clean(text_name):
    text = open(text_name,'r',encoding='utf-8').read()
    text = text.replace('\n',' ')   
    text = text.replace('  ',' ')
    tokenized_sentence = nltk.sent_tokenize(text)
    tokenized_words = [nltk.word_tokenize(sentence) for sentence in tokenized_sentence]
    tagged_words = [nltk.pos_tag(word) for word in tokenized_words]
    word_tree = [chunker.parse(word) for word in tagged_words]
    
    return tokenized_sentence,word_tree

In [ ]:
def sentences_with_relations_extraction(text):
    text_data = pd.DataFrame(text)
    text_data['times'] = 0 
    text_data['names'] = '' 
    text_data.columns = ['sentence','times','names']
    
    for j  in range(len(text_data['sentence'])):
        sentence = text_data['sentence'][j]
        words = nltk.word_tokenize(sentence)
        for i in range (len(words)):
            if (words[i] in list(name_list['name'])) and (words[i] not in text_data['names'][j]):
                text_data['times'][j] = text_data['times'][j] + 1
                if text_data['times'][j] == 1:
                    text_data['names'][j] = []
                    text_data['names'][j].append((words[i]))
                elif text_data['times'][j] > 1:
                    text_data['names'][j].append(words[i])
    text_with_names = text_data.loc[text_data['times']>1]
    text_with_names = text_with_names.reset_index(drop=True)
    return(text_with_names)

### Compose the sentenses with the names

In [ ]:
def text_relation_extraction(text_way):
    text,text_tree = data_clean(text_way)
    sentences_with_relations_extraction_now = sentences_with_relations_extraction(text)
    return(sentences_with_relations_extraction_now)

In [ ]:
sentences_with_relations_extraction_Hobbit = text_relation_extraction('The Hobbit.txt')
sentences_with_relations_extraction_Rings = text_relation_extraction('The Lord of the Rings.txt')

In [ ]:
sentences_with_relations_extraction_Hobbit['book']='Hobbits'
sentences_with_relations_extraction_Rings['book']='Ring'

# Create The Relationship Tuple

In [ ]:
r_h_relations = pd.concat([sentences_with_relations_extraction_Hobbit,sentences_with_relations_extraction_Rings])
r_h_relations = r_h_relations.reset_index(drop=True)
r_h_relations['code'] = r_h_relations.index

In [ ]:
r_h_relations['names']=r_h_relations['names'].tolist()

In [ ]:
duple_relationship=pd.DataFrame()

for i in range(len(r_h_relations)):
    if i == 0:
        a = pd.DataFrame((list(combinations(r_h_relations['names'][i],2))))
        a['sentences_code'] = i 
        a['book']=r_h_relations['book'][i]
        #a['relationship']=r_h_relations['relationship'][i]
        duple_relationship = a
        
    else :
        a = pd.DataFrame(duple_relationship)
        b = pd.DataFrame((list(combinations(r_h_relations['names'][i],2))))
        b['sentences_code'] = i 
        b['book']=r_h_relations['book'][i]
#b['relationship']=r_h_relations['relationship'][i]
        duple_relationship = pd.concat([a,b])   

In [ ]:
duple_relationship.columns=['c1','c2','sentence_code','book']
duple_relationship

In [ ]:
a = pd.merge(name_list,duple_relationship,left_on='name',right_on='c1')
c1_c2 = pd.merge(name_list,a,left_on='name',right_on='c2')
c1_c2.columns=['c2_name_code', 'c2_race', 'c2_name', 'c1_name_code', 'c1_race', 'c1_name',
       'c1', 'c2', 'sentence_code','book']
del c1_c2['c2_name']
del c1_c2['c1_name']
c1_c2 = c1_c2.sort_values(by='sentence_code')
c1_c2 = c1_c2.reset_index(drop=True)
c1_c2

In [ ]:
c1_c2['relationship'] = ''
for i in range (len(c1_c2)):
    #same one
    if c1_c2['c1_name_code'][i] == c1_c2 ['c2_name_code'][i]:
        c1_c2['relationship'][i] = 'same_people'
    else:
        continue

In [ ]:
c1_c2['relationship'].describe()

#### classify with the races ( the results have stored in the c1_c2_done

## create the relationship dataset for the model

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from load_data import get_train_test_pd
#from tensorflow.keras.utils import to_categorical
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense
from bert import modeling
from albert_zh.extract_feature import BertVector

#from att import Attention
from keras.layers import GRU, Bidirectional
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate
from tensorflow.keras.layers import Layer

from torch.utils.data import _utils
import warnings
warnings. filterwarnings('ignore')

In [ ]:
r_h_relations.to_excel('sentence_name.xlsx')
c1_c2 = pd.read_excel('c1_c2_done.xlsx', index_col=0)
sentence = pd.read_excel('sentence_name.xlsx',index_col=0)
c1_c2=pd.merge(c1_c2,sentence,how ='left',left_on='sentence_code',right_on='code')
del c1_c2['sentence_x']


In [ ]:
c1_c2.columns=['c2_name_code','c2_race','c1_name_code','c1_race','c1','c2','sentence_code','relationship','sentence','times','names','book','code']

In [ ]:
c1_c2 = c1_c2.sample(frac=1).reset_index(drop=True)
c1_c2 = c1_c2.dropna()
c1_c2 = c1_c2.reset_index(drop=True)

In [ ]:
c1_c2

In [ ]:
# -*- coding: utf-8 -*-
import json
import pandas as pd
from pprint import pprint
import bert
from transformers import BertTokenizer
tz = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
df = c1_c2
relations = list(df['relationship'].unique())
relations.remove('unknown')
relation_dict = {'unknown': 0}
relation_dict.update(dict(zip(relations, range(1, len(relations)+1))))

with open('rel_dict.json', 'w', encoding='utf-8') as h:
    h.write(json.dumps(relation_dict, ensure_ascii=False, indent=2))

pprint(df['relationship'].value_counts())
df['rel'] = df['relationship'].apply(lambda x: relation_dict[x])

texts = []
for per1, per2,r1,r2, text in zip(df['c1'].tolist(), df['c2'].tolist(),df['c1_race'].tolist(), df['c2_race'].tolist(), df['sentence'].tolist()):
    text = '$ '.join([per1, per2,r1,r2, text])
    texts.append(text)

df['text'] = texts

train_df = df.sample(frac=0.8, random_state=1024)
test_df = df.drop(train_df.index)

with open('train.txt', 'w', encoding='utf-8') as f:
    for text, rel in zip(train_df['text'].tolist(), train_df['rel'].tolist()):
        f.write(str(rel)+' '+text+'\n')

with open('test.txt', 'w', encoding='utf-8') as g:
    for text, rel in zip(test_df['text'].tolist(), test_df['rel'].tolist()):
        g.write(str(rel)+' '+text+'\n')

In [ ]:
df = c1_c2
relations = list(df['relationship'].unique())
relations.remove('unknown')
relation_dict = {'unknown': 0}
relation_dict.update(dict(zip(relations, range(1, len(relations)+1))))

with open('rel_dict.json', 'w', encoding='utf-8') as h:
    h.write(json.dumps(relation_dict, ensure_ascii=False, indent=2))

pprint(df['relationship'].value_counts())
df['rel'] = df['relationship'].apply(lambda x: relation_dict[x])

texts = []
for per1, per2,r1,r2, text in zip(df['c1'].tolist(), df['c2'].tolist(),df['c1_race'].tolist(), df['c2_race'].tolist(), df['sentence'].tolist()):
    text = '$ '.join([per1, per2,r1,r2, text])
    texts.append(text)

df['text'] = texts

train_df = df.sample(frac=0.8, random_state=1024)
test_df = df.drop(train_df.index)

with open('train.txt', 'w', encoding='utf-8') as f:
    for text, rel in zip(train_df['text'].tolist(), train_df['rel'].tolist()):
        f.write(str(rel)+' '+text+'\n')

with open('test.txt', 'w', encoding='utf-8') as g:
    for text, rel in zip(test_df['text'].tolist(), test_df['rel'].tolist()):
        g.write(str(rel)+' '+text+'\n')

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd

# 
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = [_.strip() for _ in f.readlines()]

    labels, texts = [], []
    for line in content:
        parts = line.split()
        label, text = parts[0], ' '.join(parts[1:])
        labels.append(label)
        texts.append(text)

    return labels, texts

#
def get_train_test_pd():
    file_path = 'train.txt'
    labels, texts = read_txt_file(file_path)
    train_df = pd.DataFrame({'label': labels, 'text': texts})

    file_path = 'test.txt'
    labels, texts = read_txt_file(file_path)
    test_df = pd.DataFrame({'label': labels, 'text': texts})

    return train_df, test_df


if __name__ == '__main__':

    train_df, test_df = get_train_test_pd()
    print(train_df.head())
    print(test_df.head())

    train_df['text_len'] = train_df['text'].apply(lambda x: len(tz.tokenize(x)))
    print(train_df.describe())

In [ ]:
def fill_padding(data, max_len): 
    if len(data) < max_len:
        pad_len = max_len - len(data)
        padding = [0 for _ in range(pad_len)]
        data = torch.tensor(data + padding )
    else:
        data = torch.tensor(data [:mapax_len])
    return data

In [ ]:
train_df['label'] = train_df['label'].astype(int)
test_df['label'] = test_df['label'].astype(int)

In [ ]:
train_df.to_excel('train_df.xlsx')
test_df.to_excel('test_df.xlsx')

## DistilBERT Model

In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import *

In [ ]:
all_data = pd.read_excel('train_df.xlsx',index_col=0)
test_df = pd.read_excel('test_df.xlsx',index_col=0)
train_df = all_data.iloc[:1882,:]
eval_df =  all_data.iloc[1882:,:]

In [ ]:
test_df.to_excel('validation_set.xlsx')
train_df.to_excel('train_set.xlsx')
eval_df.to_excel('test_set.xlsx')

In [ ]:
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)
eval_data = Dataset.from_pandas(eval_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,padding=True)

In [ ]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_eval = eval_data.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=20)

In [ ]:
from datasets import load_metric
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

train_out = trainer.train()

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
prediction = classifier(test_data['text'])

In [ ]:
pred_label=[]
for i in range(len(prediction)):
    pred_label.append(int(prediction[i]['label'][-1]))

In [ ]:
import sklearn.metrics as metrics
metrics.f1_score(pred_label, test_data['label'], average='weighted')

In [ ]:
a=0
for i in range(len(prediction)):
    if test_data['label'][i] == pred_label[i]:
        a=a+1

## LSTM and TextCNN with Word2Vec

In [ ]:
from gensim.models import word2vec
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, Input
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D
from keras.layers import  BatchNormalization
from keras.layers import Convolution1D, Conv1D,MaxPooling1D
from keras.layers import Dense, Embedding, Input, Lambda, Reshape
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
#from keras.utils import to_categorical
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import jieba as jb
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import  pad_sequences
from keras .models import  Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import  Conv1D
from keras.layers.convolutional import  MaxPooling1D
import keras
from keras .models import load_model
from tensorflow.keras.utils import to_categorical
from nltk import word_tokenize     
import gensim
from keras.layers import Input
from keras.layers import *
from keras.models import Model
import sklearn.metrics as metrics

In [ ]:
f=open("train.txt",encoding='utf-8')
sentences = f.readlines() 
f.close()
w2v_model =Word2Vec(sentences, hs=1, min_count=1, window=5, vector_size=500)
w2v_model.wv.save_word2vec_format("./word2Vec" + ".pkl", binary=True)

In [ ]:
train_df = pd.read_excel('train_set.xlsx',index_col=0)
test_df = pd.read_excel('test_set.xlsx',index_col=0)
eval_df = pd.read_excel('validation_set.xlsx',index_col=0)
all_train = pd.concat([train_df,eval_df])

In [ ]:
X_train=encode_docs(tokenizer,512,all_train['text'])
y_train=all_train['label']
y_train=np.array(y_train)#(130583, 1)
Y_train=keras.utils.np_utils.to_categorical(y_train,20)#
X_test=encode_docs(tokenizer,512,test_df['text'])
y_test=test_df['label']
y_test=np.array(y_test)#(130583, 1)
Y_test=keras.utils.np_utils.to_categorical(y_test,20)#
vocad_size=len(tokenizer.word_index)+1

In [ ]:
tokenizer=Tokenizer()

### TextCNN

In [ ]:
def TextCNN_model_1(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
    main_input = Input(shape=(512,), dtype='float64')
    
    embedder = (Embedding(vocad_size, 100, input_length=50))
    embed = embedder(main_input)
    
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=49)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=48)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=47)(cnn3)
   
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(20, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
    one_hot_labels =one_hot_labels = keras.utils.np_utils.to_categorical(y_train,20) 
    model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=20)
    #y_test_onehot = keras.utils.to_categorical(y_test, num_classes=3)
    return model

In [ ]:
embedding_matrix = np.zeros((len(vocab) + 1, 500))
for word, i in vocab.items():
    try:
        embedding_vector = w2v_model.wv[str(word)]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue
 

def TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix):
    
    main_input = Input(shape=(512,), dtype='float64')
    
    embedder = Embedding(len(vocab) + 1, 500, input_length=512, weights=[embedding_matrix], trainable=False)
    #embedder = Embedding(len(vocab) + 1, 300, input_length=50, trainable=False)
    embed = embedder(main_input)
    
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=49)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=48)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=47)(cnn3)
    
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(20, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
    one_hot_labels = keras.utils.np_utils.to_categorical(y_train,20) 
    model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=20)
    return model

In [ ]:
def encode_docs(tokenizer,max_length,docs):
    encoded=tokenizer.texts_to_sequences(docs)
    padded=pad_sequences(encoded,maxlen=max_length,padding='post')
    return padded

In [ ]:
textcnnw2v = TextCNN_model_2(X_train, y_train, X_test, y_test,embedding_matrix)

In [ ]:
## textcnnw2v
y_predict = textcnnw2v.predict(X_test) 
y_predict = np.argmax(y_predict,axis=1)
test_y=np.array(y_test)
print('accuracy', metrics.accuracy_score(y_test, y_predict))
print('f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

In [ ]:
textcnn = TextCNN_model_1(X_train, y_train, X_test, y_test)

In [ ]:
# textcnn
y_predict = textcnn.predict(X_test)  
y_predict = np.argmax(y_predict,axis=1)
test_y=np.array(y_test)
print('accuracy', metrics.accuracy_score(y_test, y_predict))
print('f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

### LSTM

In [ ]:

vocab_dim = 500 
maxlen = 512 
batch_size = 100 
n_epoch = 20   

In [ ]:
X_train=encode_docs(tokenizer,512,train_df['text'])
y_train=train_df['label']
y_train=np.array(y_train)#(130583, 1)
Y_train=keras.utils.np_utils.to_categorical(y_train,20)#

X_eval=encode_docs(tokenizer,512,eval_df['text'])
y_eval=eval_df['label']
y_eval=np.array(y_eval)#(130583, 1)
Y_eval=keras.utils.np_utils.to_categorical(y_eval,20)#
#vocad_size=len(tokenizer.word_index)+1

X_test=encode_docs(tokenizer,512,test_df['text'])
y_test=test_df['label']
y_test=np.array(y_test)#(130583, 1)
Y_test=keras.utils.np_utils.to_categorical(y_test,20)#
vocad_size=len(tokenizer.word_index)+1

In [ ]:
def train_lstm(p_n_symbols, p_embedding_weights, p_X_train, p_y_train, p_X_eval, p_y_eval, p_X_test, p_y_test
):
    print('创建模型...')
    model = Sequential()
    model.add(Embedding(output_dim=vocab_dim,  
                        input_dim=p_n_symbols, 
                        mask_zero=True,         
                        weights=[p_embedding_weights],   
                        input_length=maxlen ))      
    
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2 ))
    model.add(Dense(units=20,  
                    activation='softmax'))
    model.summary()

    print('modeling...')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print("training...")
    train_label_one_hot = keras.utils.np_utils.to_categorical(p_y_train,20)  
    test_label_one_hot = keras.utils.np_utils.to_categorical(p_y_test,20)  
    eval_label_one_hot = keras.utils.np_utils.to_categorical(p_y_eval,20) 
    
    train_history = model.fit(p_X_train, train_label_one_hot, batch_size=batch_size, epochs=n_epoch,
              validation_data=(p_X_eval, eval_label_one_hot))
    return train_history,model


In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
his_lstm,lstm =  train_lstm(4534,embedding_matrix,X_train, y_train, X_eval, y_eval, X_test, y_test)

In [ ]:
## textcnn
y_predict = lstm.predict(X_test) 
y_predict = np.argmax(y_predict,axis=1)
test_y=np.array(y_test)
print('accuray', metrics.accuracy_score(y_test, y_predict))
print('f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

## Visualization with neo4j

In [ ]:
import pandas as pd
import numpy as np
import csv
import py2neo
from py2neo import Graph,Node,Relationship,NodeMatcher

In [ ]:
data = pd.read_excel('neo4j.xlsx',index_col=0)

In [ ]:
del data['c2_name_code']
del data['c1_name_code']
del data['sentence_code']
del data['sentence']
data=data.reset_index(drop=1)
data = data.drop_duplicates()
data=data.reset_index(drop=1)

In [ ]:
h_b=[]
r_b=[]
for i in range (len(data)):
    if data['book'][i]=='Ring':
        r_b.append(data['c1'][i])
        r_b.append(data['c2'][i])
    else:
        h_b.append(data['c1'][i])
        h_b.append(data['c2'][i])
both_list = ['Bofur','Dwalin','Smaug','Thorin',     
'Fundin','Bombur','Dori','Bill','Bilbo','Bifur','Gandalf','Tom'     ,   'Beorn'    ,  'Bard'      , 
'Balin'      ,'Elrond'    ,'Ori'        ,'Gollum'     ]
only_h_list=list(set(h_b)-set(both_list))
only_r_list=list(set(r_b)-set(both_list))

In [ ]:
data['c1_book']=''
data['c2_book']=''
for i in range(len(data)):
    if data['c1'][i] in both_list:
        data['c1_book'][i] = 'both'
       # print(1)
    elif data['c1'][i] in only_h_list:
        data['c1_book'][i] = 'only_hobbits'
       # print(2)
    elif data['c1'][i] in only_r_list:
        data['c1_book'][i] = 'only_ring'
for i in range(len(data)):        
    if data['c2'][i] in both_list:
        data['c2_book'][i] = 'both'
    if data['c2'][i] in only_h_list:
        data['c2_book'][i] = 'only_hobbits'
    if data['c2'][i] in only_r_list:
        data['c2_book'][i] = 'only_ring'

In [ ]:
g=Graph('http://localhost:7474',user='',password='')
g.delete_all()

In [ ]:
for i in range(len(data)):
    item = data.iloc[i,:]
    start_node=Node(item[5],  name=item[2], race=item[1])
    end_node = Node(item[5],  name=item[3], race=item[0])
    relation=Relationship(start_node,item[4],end_node)
    g.merge(start_node,"race","name")
    g.merge(end_node, "race", "name")
    g.merge(relation, "race", "name")

In [ ]:
for i in range(len(data)):
    item = data.iloc[i,:]
    start_node=Node(item[6], name=item[2], race=item[1])
    end_node = Node(item[6], name=item[3], race=item[0])
    relation=Relationship(start_node,item[4],end_node)
    g.merge(start_node,"book","name")
    g.merge(end_node, "book", "name")
    g.merge(relation, "book", "name")